In [1]:
!pip3 install farm-haystack

     |████████████████████████████████| 81kB 2.1MB/s 
     |████████████████████████████████| 51kB 5.2MB/s 
     |████████████████████████████████| 983kB 15.6MB/s 
     |████████████████████████████████| 5.5MB 24.2MB/s 
     |████████████████████████████████| 143kB 32.5MB/s 
     |████████████████████████████████| 81kB 8.3MB/s 
     |████████████████████████████████| 194kB 35.0MB/s 
     |████████████████████████████████| 225kB 44.7MB/s 
     |████████████████████████████████| 3.0MB 44.1MB/s 
     |████████████████████████████████| 7.9MB 32.9MB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
     |████████████████████████████████| 317kB 37.2MB/s 
     |████████████████████████████████| 92kB 7.9MB/s 
     |████████████████████████████████| 61kB 6.3MB/s 
     |████████████████████████████████| 327kB 36.3MB/s 
     |████████████████████████████████| 2.5MB 43.9MB/s 
     |████████████████████████████████| 890kB 43.2MB/s 
     |████████████████████████████████| 47.7MB 117kB/s 
    

In [2]:
!pip3 install torch==1.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [4]:
from haystack import Finder
from haystack.preprocessor.cleaning import clean_wiki_text
from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers

Question answering with Elastic search

In [5]:

! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.6.2-linux-x86_64.tar.gz -q

In [6]:
! tar -xzf elasticsearch-7.6.2-linux-x86_64.tar.gz

In [7]:
! chown -R daemon:daemon elasticsearch-7.6.2

In [8]:
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.6.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 30

In [9]:

from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

11/03/2020 08:18:22 - INFO - elasticsearch -   PUT http://localhost:9200/document [status:200 request:0.497s]
11/03/2020 08:18:23 - INFO - elasticsearch -   PUT http://localhost:9200/label [status:200 request:0.259s]


In [10]:
doc_dir = "data/article_txt_got"
s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt.zip"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

dicts = convert_files_to_dicts(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)


print(dicts[:3])

document_store.write_documents(dicts)


11/03/2020 08:19:24 - INFO - haystack.preprocessor.utils -   Fetching from https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt.zip to `data/article_txt_got`
100%|██████████| 1095120/1095120 [00:00<00:00, 1799789.51B/s]


[{'text': '"\'\'\'The Prince of Winterfell\'\'\'" is the eighth episode of the second season of HBO\'s medieval fantasy television series \'\'Game of Thrones\'\'. The episode is written by series co-creators David Benioff and D. B. Weiss and directed, for the third time in this season, by Alan Taylor. It premiered on May 20, 2012.\nThe title of the episode refers to Theon Greyjoy as ruler of Winterfell after disposing of the Stark children.', 'meta': {'name': '74_The_Prince_of_Winterfell.txt'}}, {'text': "\n===In King's Landing===\nTyrion and Bronn plan the defense of King's Landing with the aid of old texts.\nWhen Tyrion is accused of plotting to kill King Joffrey, Cersei mistakenly kidnaps Ros instead of Shae, and Tyrion swears to Cersei that she will pay for her actions.\nJoffrey's inexperience and arrogance leave Tyrion fearful for the coming battle. Varys informs Tyrion that Daenerys is alive with three dragons.", 'meta': {'name': '74_The_Prince_of_Winterfell.txt'}}, {'text': "\n=

11/03/2020 08:19:26 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.157s]
11/03/2020 08:19:27 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.024s]
11/03/2020 08:19:29 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.126s]
11/03/2020 08:19:30 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.025s]
11/03/2020 08:19:31 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.008s]


In [11]:
from haystack.retriever.sparse import ElasticsearchRetriever
retriever = ElasticsearchRetriever(document_store=document_store)

In [12]:
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=False)

11/03/2020 08:21:16 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None
11/03/2020 08:21:16 - INFO - farm.infer -   Could not find `deepset/roberta-base-squad2` locally. Try to download from model hub ...
11/03/2020 08:21:16 - INFO - filelock -   Lock 139857455548792 acquired on /root/.cache/torch/transformers/f7d4b9379a9c487fa03ccf3d8e00058faa9d664cf01fc03409138246f48760da.c6288e0f84ec797ba5c525c923a5bbc479b47c761aded9734a5f6a473b044c8d.lock


11/03/2020 08:21:16 - INFO - filelock -   Lock 139857455548792 released on /root/.cache/torch/transformers/f7d4b9379a9c487fa03ccf3d8e00058faa9d664cf01fc03409138246f48760da.c6288e0f84ec797ba5c525c923a5bbc479b47c761aded9734a5f6a473b044c8d.lock
11/03/2020 08:21:16 - INFO - filelock -   Lock 139857455549856 acquired on /root/.cache/torch/transformers/8c0c8b6371111ac5fbc176aefcf9dbe129db7be654c569b8375dd3712fc4dc67.d045adc91e17ecdf7dc3eeff4c875df94bdf2eb749d72b3ae47ae93f8e85213c.lock


11/03/2020 08:21:34 - INFO - filelock -   Lock 139857455549856 released on /root/.cache/torch/transformers/8c0c8b6371111ac5fbc176aefcf9dbe129db7be654c569b8375dd3712fc4dc67.d045adc91e17ecdf7dc3eeff4c875df94bdf2eb749d72b3ae47ae93f8e85213c.lock


11/03/2020 08:21:40 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
11/03/2020 08:21:48 - WARNING - farm.modeling.prediction_head -   Some unused parameters are passed to the QuestionAnsweringHead. Might not be a problem. Params: {"loss_ignore_index": -1}
11/03/2020 08:21:48 - INFO - filelock -   Lock 139857455549408 acquired on /root/.cache/torch/transformers/1e3af82648d7190d959a9d76d727ef629b1ca51b3da6ad04039122453cb56307.6a4061e8fc00057d21d80413635a86fdcf55b6e7594ad9e25257d2f99a02f4be.lock


11/03/2020 08:21:49 - INFO - filelock -   Lock 139857455549408 released on /root/.cache/torch/transformers/1e3af82648d7190d959a9d76d727ef629b1ca51b3da6ad04039122453cb56307.6a4061e8fc00057d21d80413635a86fdcf55b6e7594ad9e25257d2f99a02f4be.lock
11/03/2020 08:21:49 - INFO - filelock -   Lock 139857455549408 acquired on /root/.cache/torch/transformers/b901c69e8e7da4a24c635ad81d016d274f174261f4f5c144e43f4b00e242c3b0.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda.lock


11/03/2020 08:21:49 - INFO - filelock -   Lock 139857455549408 released on /root/.cache/torch/transformers/b901c69e8e7da4a24c635ad81d016d274f174261f4f5c144e43f4b00e242c3b0.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda.lock
11/03/2020 08:21:49 - INFO - filelock -   Lock 139857454575512 acquired on /root/.cache/torch/transformers/2d9b03b59a8af464bf4238025a3cf0e5a340b9d0ba77400011e23c130b452510.16f949018cf247a2ea7465a74ca9a292212875e5fd72f969e0807011e7f192e4.lock


11/03/2020 08:21:49 - INFO - filelock -   Lock 139857454575512 released on /root/.cache/torch/transformers/2d9b03b59a8af464bf4238025a3cf0e5a340b9d0ba77400011e23c130b452510.16f949018cf247a2ea7465a74ca9a292212875e5fd72f969e0807011e7f192e4.lock
11/03/2020 08:21:49 - INFO - filelock -   Lock 139857454575512 acquired on /root/.cache/torch/transformers/507984f2e28c7dfed5db9a20acd68beb969c7f2833abc9e582e967fa0291f3dc.100c88dbe27dbd73822c575274ade4eb2427596ac56e96769249b7512341654d.lock


11/03/2020 08:21:49 - INFO - filelock -   Lock 139857454575512 released on /root/.cache/torch/transformers/507984f2e28c7dfed5db9a20acd68beb969c7f2833abc9e582e967fa0291f3dc.100c88dbe27dbd73822c575274ade4eb2427596ac56e96769249b7512341654d.lock


11/03/2020 08:21:50 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None
11/03/2020 08:21:50 - INFO - farm.infer -   Got ya 1 parallel workers to do inference ...
11/03/2020 08:21:50 - INFO - farm.infer -    0 
11/03/2020 08:21:50 - INFO - farm.infer -   /w\
11/03/2020 08:21:50 - INFO - farm.infer -   /'\
11/03/2020 08:21:50 - INFO - farm.infer -   


In [13]:
finder = Finder(reader, retriever)

In [27]:
prediction = finder.get_answers(question="Who is the education minister", top_k_retriever=10, top_k_reader=5)

11/03/2020 09:22:14 - INFO - haystack.finder -   Reader is looking for detailed answer in 40651 chars ...
Inferencing Samples: 100%|██████████| 1/1 [00:06<00:00,  6.88s/ Batches]


In [30]:
print_answers(prediction, details="minimal")

[   {   'answer': 'Magister Illyrio',
        'context': 'erys. He is undefeated in battle. Viserys Targaryen '
                   'conspired with Magister Illyrio to marry Daenerys to Drogo '
                   'in order to get his support in an invas'},
    {   'answer': 'Roger Allam Illyrio Mopatis',
        'context': '\n'
                   "* '''Illyrio Mopatis''' (season 1) portrayed by Roger "
                   'Allam. Roger Allam Illyrio Mopatis is a wealthy magister '
                   'of the free city of Pentos. Illiyrio h'},
    {   'answer': 'Yezzan',
        'context': "al assault on Meereen. When Daenerys's dragons defeat the "
                   'slaver fleet, Yezzan is betrayed by Razdal and Belicho and '
                   'handed over to be killed by Grey '},
    {   'answer': 'Prince Doran',
        'context': ", she has become a part of Ellaria Sand's plans for "
                   'revenge although Prince Doran refuses that the young '
                   'princ

Question answering without Elastic search

In [19]:
from haystack.document_store.memory import InMemoryDocumentStore
document_store = InMemoryDocumentStore()

In [20]:
doc_dir = "data/article_txt_got"
s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt.zip"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

dicts = convert_files_to_dicts(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)


print(dicts[:3])
document_store.write_documents(dicts)


11/03/2020 08:52:43 - INFO - haystack.preprocessor.utils -   Found data stored in `data/article_txt_got`. Delete this first if you really want to fetch new data.


[{'text': '"\'\'\'The Prince of Winterfell\'\'\'" is the eighth episode of the second season of HBO\'s medieval fantasy television series \'\'Game of Thrones\'\'. The episode is written by series co-creators David Benioff and D. B. Weiss and directed, for the third time in this season, by Alan Taylor. It premiered on May 20, 2012.\nThe title of the episode refers to Theon Greyjoy as ruler of Winterfell after disposing of the Stark children.', 'meta': {'name': '74_The_Prince_of_Winterfell.txt'}}, {'text': "\n===In King's Landing===\nTyrion and Bronn plan the defense of King's Landing with the aid of old texts.\nWhen Tyrion is accused of plotting to kill King Joffrey, Cersei mistakenly kidnaps Ros instead of Shae, and Tyrion swears to Cersei that she will pay for her actions.\nJoffrey's inexperience and arrogance leave Tyrion fearful for the coming battle. Varys informs Tyrion that Daenerys is alive with three dragons.", 'meta': {'name': '74_The_Prince_of_Winterfell.txt'}}, {'text': "\n=

In [21]:
from haystack.retriever.sparse import TfidfRetriever
retriever = TfidfRetriever(document_store=document_store)

11/03/2020 08:54:13 - INFO - haystack.retriever.sparse -   Found 2497 candidate paragraphs from 2497 docs in DB


In [22]:
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=False)

11/03/2020 08:55:17 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None
11/03/2020 08:55:17 - INFO - farm.infer -   Could not find `deepset/roberta-base-squad2` locally. Try to download from model hub ...
11/03/2020 08:55:24 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
11/03/2020 08:55:31 - WARNING - farm.modeling.prediction_head -   Some unused parameters are passed to the QuestionAnsweringHead. Might not be a problem. Params: {"loss_ignore_index": -1}
11/03/2020 08:55:32 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None
11/03/2020 08:55:32 - INFO - farm.infer -   Got ya 1 parallel workers to do inference ...
11/03/2020 08:55:32 - INFO - farm.infer -    0 
11/03/2020 08:55:

In [23]:
finder = Finder(reader, retriever)

In [26]:
prediction = finder.get_answers(question="Who is the father of computer?", top_k_retriever=10, top_k_reader=5)

11/03/2020 09:21:30 - INFO - haystack.finder -   Reader is looking for detailed answer in 26714 chars ...
Inferencing Samples: 100%|██████████| 1/1 [00:01<00:00,  1.79s/ Batches]


In [29]:
print_answers(prediction, details="minimal")

[   {   'answer': 'Magister Illyrio',
        'context': 'erys. He is undefeated in battle. Viserys Targaryen '
                   'conspired with Magister Illyrio to marry Daenerys to Drogo '
                   'in order to get his support in an invas'},
    {   'answer': 'Roger Allam Illyrio Mopatis',
        'context': '\n'
                   "* '''Illyrio Mopatis''' (season 1) portrayed by Roger "
                   'Allam. Roger Allam Illyrio Mopatis is a wealthy magister '
                   'of the free city of Pentos. Illiyrio h'},
    {   'answer': 'Yezzan',
        'context': "al assault on Meereen. When Daenerys's dragons defeat the "
                   'slaver fleet, Yezzan is betrayed by Razdal and Belicho and '
                   'handed over to be killed by Grey '},
    {   'answer': 'Prince Doran',
        'context': ", she has become a part of Ellaria Sand's plans for "
                   'revenge although Prince Doran refuses that the young '
                   'princ